<a href="https://colab.research.google.com/github/harshyadav1508/GenAiPoc/blob/main/langchainv0.1.0/langchainRAGIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install langchain
! pip install langchain-openai
! pip install beautifulsoup4
! pip install faiss-cpu

In [6]:
import os
import openai
from langchain_openai import AzureChatOpenAI, ChatOpenAI

In [7]:
os.environ["AZURE_OPENAI_API_KEY"] = "***"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://caip-azure-openai.openai.azure.com/'
os.environ["OPENAI_API_VERSION"]    = "2023-07-01-preview"

In [8]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4",
    openai_api_type="azure"
)


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langchain-v0-1-0/")

docs = loader.load()

In [11]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [12]:
from langchain_openai import AzureOpenAIEmbeddings

os.environ["AZURE_OPENAI_API_KEY"] = "***"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://caip-openai-dev.openai.azure.com/"

embeddings = AzureOpenAIEmbeddings(

    # azure_deployment="text-davinci-002",
    openai_api_type="azure",
    openai_api_version="2023-07-01-preview",
    )
vectorstore = FAISS.from_documents(documents,embeddings)

In [14]:
# create chain for docs
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


template="""
Answer the following question based only on the provided context:
<context>{context}</context>

Question:{input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain =create_stuff_documents_chain(llm,prompt)

In [15]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
response = retrieval_chain.invoke({
    "input": "what is new in langchain 0.1.0"
})
response["answer"]

'LangChain 0.1.0 is the first stable version that is fully backwards compatible and comes in both Python and JavaScript. It features improved focus through both functionality and documentation and introduces a new versioning standard. This release aims to earn developer trust and allows for the systematic and safe evolution of the library.'

In [17]:
# conversational retrieval chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

In [19]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [20]:
# simulate conversation history

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [21]:
response['answer']

'LangChain 0.1.0 is the first stable version and brings several improvements and changes:\n\n1. Fully backwards compatible: This version maintains compatibility with previous versions, making it easier for developers to upgrade without worrying about breaking changes.\n2. Available in both Python and JavaScript: LangChain 0.1.0 has been developed for both Python and JavaScript, ensuring a broader reach for developers.\n3. Improved focus through functionality and documentation: The new version comes with enhanced features and better documentation, making it more user-friendly.\n4. New versioning standard: From 0.1.0 onwards, breaking changes to the public API will result in a minor version bump, and bug fixes or new features will result in a patch version bump. This will help developers update with confidence and allow the LangChain team to deprecate and delete old code more responsibly.\n\nOther notable changes include architectural updates, separating langchain-core and partner packag